# Using a Jupyter Notebook to perform a CoDa Contradiction Nanopublication Analysis (with 'r' measure)

Imran Asif ([ORCID:0000-0002-1144-6265](https://orcid.org/0000-0002-1144-6265))  
PhD Student, Computer Science

_Heriot-Watt University, Edinburgh, UK_

In [1]:
import ipywidgets as widgets
from SPARQLWrapper import SPARQLWrapper, JSON
import time
import seaborn as sns
# import pandas
import pandas as pd
# import matplotlib
import matplotlib.pyplot as plt

from IPython.display import display
from IPython.display import clear_output
from IPython.core.display import HTML
from IPython.display import Javascript
import urllib.parse
import statistics

import requests
from datetime import datetime

### You need to give the SPARQL endpoint to run the script

In [2]:
SPARQL_endpoint = "http://localhost:3030/nanopubs_r_04_11_2020/sparql"
dbSparql = SPARQLWrapper(SPARQL_endpoint)
dbSparql.setReturnFormat(JSON)
def queryCoDa(query):
    dbSparql.setQuery(query)
    results = dbSparql.queryAndConvert()
    return results

# Define Methods to Get Assertions, Provenance and IVs

In [3]:
def getAllAssertion():
    #The followig query get all assertion information. Query result contains the concatenate of IVs with "," 
    query = """
    prefix np: <http://www.nanopub.org/nschema#>
    PREFIX cc: <https://data.cooperationdatabank.org/vocab/class/>
    PREFIX cp: <https://data.cooperationdatabank.org/vocab/prop/>
    PREFIX cr: <https://data.cooperationdatabank.org/id/>
    PREFIX n: <https://data.cooperationdatabank.org/coda/nanopub/>

    select ?np ?obs ?a ?effectValue ?dv ?iv ?var ?sampleSize ?stdErr where {
      graph ?h {
        ?np a np:Nanopublication ;
          np:hasAssertion ?a .
      }
      graph ?a {
                    ?obs <http://www.example.org/claimsDV> ?dv ;
                       <http://www.example.org/claimsIV> ?iv ;
                       <http://www.example.org/has-effect-value> ?effectValue ;
                       <http://www.example.org/has-sample-size> ?sampleSize ;
                       <http://www.example.org/has-stderr> ?stdErr ;
                       <http://www.example.org/has-var> ?var ;
                }
    }"""

    return queryCoDa(query)

###### END getAllAssertion #####
#######################################################################################
#Fetch IVs from the Assertion
def getIVs():
    query = """
        prefix np: <http://www.nanopub.org/nschema#>
        PREFIX cc: <https://data.cooperationdatabank.org/vocab/class/>
        PREFIX cp: <https://data.cooperationdatabank.org/vocab/prop/>
        PREFIX cr: <https://data.cooperationdatabank.org/id/>
        PREFIX n: <https://data.cooperationdatabank.org/coda/nanopub/>

        select DISTINCT ?iv where {
          graph ?h {
            ?np a np:Nanopublication ;
              np:hasAssertion ?a ;
          }
          graph ?a {
                        ?s <http://www.example.org/claimsIV> ?iv .
                    }
        }"""

    return queryCoDa(query)

###### END IVs #####
#######################################################################################
#Get provence info by NP IRI
def getProvenanceByNP(np):
    query = """
            prefix np: <http://www.nanopub.org/nschema#>
            PREFIX cc: <https://data.cooperationdatabank.org/vocab/class/>
            PREFIX cp: <https://data.cooperationdatabank.org/vocab/prop/>
            PREFIX cr: <https://data.cooperationdatabank.org/id/>
            PREFIX n: <https://data.cooperationdatabank.org/coda/nanopub/>
            PREFIX prov: <http://www.w3.org/ns/prov#>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX dct: <http://purl.org/dc/terms/>

            select distinct ?study ?oneShot ?oneShotLabel (group_concat(distinct ?gSize;separator=',')  as ?groupSize) 
             ?matching ?matchingLabel ?gameIncentive ?gameIncentiveLabel ?Experimental ?ExperimentalLabel 
             (group_concat(distinct ?numOfChoices;separator=',')  as ?numberOfChoices) ?kIndex 
             (group_concat(distinct ?mpcr;separator=',')  as ?MPCR) ?maleProportion 
            (group_concat(distinct ?yodCollection;separator=',')  as ?yearOfDataCollection) (count(distinct ?creator) as ?authorCount)
            ?doi ?date
            where {
              graph ?h {
                <"""+np+"""> a np:Nanopublication ;
                      np:hasProvenance ?prov .
              }
              graph ?prov {
                        ?a  prov:wasDerivedFrom ?study .
                        OPTIONAL {?a dct:creator ?creator . }
                        OPTIONAL {?a cp:doi ?doi .}
                        OPTIONAL {?a dct:date ?date . }
                        OPTIONAL {?a cp:studyOneShot ?oneShot .
                                  ?oneShot rdfs:label ?oneShotLabel . }
                        OPTIONAL {?a cp:studyMatchingProtocol ?matching .
                                  ?matching  rdfs:label ?matchingLabel . }
                        OPTIONAL {?a cp:studyGameIncentive ?gameIncentive . 
                                  ?gameIncentive  rdfs:label ?gameIncentiveLabel . }
                        OPTIONAL {?a cp:studyExperimentalSetting ?Experimental . 
                                  ?Experimental  rdfs:label ?ExperimentalLabel . }
                        OPTIONAL {?a cp:yearOfDataCollection ?yodCollection . }
                        OPTIONAL {?a cp:studyNumberOfChoices ?numOfChoices .}
                        OPTIONAL {?a cp:studyGroupSize ?gSize .}
                        OPTIONAL { ?a cp:studyKindex ?kIndex . }
                        OPTIONAL { ?a cp:studyMPCR ?mpcr . }
                        OPTIONAL { ?a cp:maleProportion ?maleProportion . } 
              }
            }
            GROUP BY ?study ?oneShot ?oneShotLabel ?matching ?matchingLabel ?gameIncentive ?gameIncentiveLabel 
                     ?Experimental ?ExperimentalLabel ?kIndex ?maleProportion ?date ?doi"""
            
    resultProv = queryCoDa(query)
    return resultProv

# Pre-Processing 

In [4]:
resultAssertion = getAllAssertion()
resultIVs = getIVs()

In [5]:
def getAuthorFromDOI(doi):
    author = 'No Author'
    # Making a get request
    try:
        if doi == 'http://dx.doi.org/NA':
            author = 'No Author'
        else:    
            #https://api.openaccessbutton.org/find?id=10.1016/j.socec.2016.04.002
            doi_id = doi.split('.org/')[1]
            #r = requests.get('https://api.datacite.org/dois/'+doi_id).json()
            #author = r['data']['attributes']['creators'][0]['familyName']

            r = requests.get('https://api.openaccessbutton.org/metadata?id='+doi_id).json()
            author = r['author'][0]['family']
    except:
        author = 'No Author'
        #print(doi)
            
    return author

In [6]:
dicIVs = {}
for result in resultIVs["results"]["bindings"]:
    splitIV = result["iv"]["value"].split("/")
    dicIVs[splitIV[len(splitIV)-1]] = {'Batch1': 0, 'Batch2': 0, 'Batch3': 0, 'Batch4': 0, 'Batch5': 0, 'Batch6': 0, 'Batch7': 0,
                                         'ObjBatch1': [], 'ObjBatch2': [], 'ObjBatch3': [], 'ObjBatch4': [], 'ObjBatch5': [],
                                         'ObjBatch6': [], 'ObjBatch7': []}

############################################################

for result in resultAssertion["results"]["bindings"]:
    
    #First Get the provence info and put into the npObj
    # Call the meethod getProvenanceByNp
    resProv = getProvenanceByNP(result["np"]["value"])
    
    #Define/assign Provenance Variables
    oneShot = ''
    groupSize = ''
    matching = ''
    gameIncentive = ''
    Experimental = ''
    numberOfChoices = ''
    kIndex = ''
    MPCR = ''
    maleProportion = ''
    yearOfDataCollection = ''
    doi = ''
    pubYear = ''
    authorCount = ''
    
    if('oneShot' in resProv['results']['bindings'][0].keys()):
        oneShot = resProv['results']['bindings'][0]['oneShotLabel']['value']
    
    if('groupSize' in resProv['results']['bindings'][0].keys()):
        groupSize = resProv['results']['bindings'][0]['groupSize']['value']
    
    if('matching' in resProv['results']['bindings'][0].keys()):
        matching = resProv['results']['bindings'][0]['matchingLabel']['value']
        
    if('gameIncentive' in resProv['results']['bindings'][0].keys()):
        gameIncentive = resProv['results']['bindings'][0]['gameIncentiveLabel']['value']
        
    if('Experimental' in resProv['results']['bindings'][0].keys()):
        Experimental = resProv['results']['bindings'][0]['ExperimentalLabel']['value']
    
    if('numberOfChoices' in resProv['results']['bindings'][0].keys()):
        numberOfChoices = resProv['results']['bindings'][0]['numberOfChoices']['value']
    
    if('kIndex' in resProv['results']['bindings'][0].keys()):
        kIndex = resProv['results']['bindings'][0]['kIndex']['value']
    
    if('MPCR' in resProv['results']['bindings'][0].keys()):
        MPCR = resProv['results']['bindings'][0]['MPCR']['value']
    
    if('maleProportion' in resProv['results']['bindings'][0].keys()):
        maleProportion = resProv['results']['bindings'][0]['maleProportion']['value']
        
    if('yearOfDataCollection' in resProv['results']['bindings'][0].keys()):
        yearOfDataCollection = resProv['results']['bindings'][0]['yearOfDataCollection']['value']
    
    try:
        if('date' in resProv['results']['bindings'][0].keys()):
            strDate = resProv['results']['bindings'][0]['date']['value'].split(":")[0]
            date = datetime.strptime(strDate, '%Y-%m-%d')
            pubYear = date.year
    except:
        d = resProv['results']['bindings'][0]['date']['value']
        #print(d)
    
    if('doi' in resProv['results']['bindings'][0].keys()):
        authorCount = int(resProv['results']['bindings'][0]['authorCount']['value'])
        doi = resProv['results']['bindings'][0]['doi']['value']
                
    npObj = {
            'np': result["np"]["value"],
            'obs': result["obs"]["value"],
            'authorCount': authorCount,
            'publicationYear': pubYear,
            'doi': doi,
            'effectValue': result["effectValue"]["value"],
            'dv': result["dv"]["value"],
            'iv': result["iv"]["value"],
            'var': result["var"]["value"],
            'sampleSize': result["sampleSize"]["value"],
            'stdErr': result["stdErr"]["value"],
            'study': resProv['results']['bindings'][0]['study']['value'],
            'oneShot': oneShot,
            'groupSize': groupSize,
            'matching': matching,
            'gameIncentive': gameIncentive,
            'Experimental': Experimental,
            'numberOfChoices': numberOfChoices,
            'kIndex': kIndex,
            'MPCR': MPCR,
            'maleProportion': maleProportion,
            'yearOfDataCollection': yearOfDataCollection
    }

    
    splitIVs = result["iv"]["value"].split("/")
    lstIVBatches = dicIVs.get(splitIVs[len(splitIVs)-1])
    
    # Rounded to three decimal places.
    if round(float(result["effectValue"]["value"]), 3) >= -1.000 and round(float(result["effectValue"]["value"]), 3) <= -0.500:
        lstIVBatches['Batch1'] = lstIVBatches['Batch1'] + 1
        lstIVBatches['ObjBatch1'].append(npObj)  
    elif round(float(result["effectValue"]["value"]), 3) >= -0.499 and round(float(result["effectValue"]["value"]), 3) <= -0.300:
        lstIVBatches['Batch2'] = lstIVBatches['Batch2'] + 1
        lstIVBatches['ObjBatch2'].append(npObj)  
    elif round(float(result["effectValue"]["value"]), 3) >= -0.299 and round(float(result["effectValue"]["value"]), 3) <= -0.051:
        lstIVBatches['Batch3'] = lstIVBatches['Batch3'] + 1
        lstIVBatches['ObjBatch3'].append(npObj)  
    elif round(float(result["effectValue"]["value"]), 3) >= -0.050 and round(float(result["effectValue"]["value"]), 3) <= 0.050:
        lstIVBatches['Batch4'] = lstIVBatches['Batch4'] + 1
        lstIVBatches['ObjBatch4'].append(npObj)  
    elif round(float(result["effectValue"]["value"]), 3) >= 0.051 and round(float(result["effectValue"]["value"]), 3) <= 0.299:
        lstIVBatches['Batch5'] = lstIVBatches['Batch5'] + 1
        lstIVBatches['ObjBatch5'].append(npObj)  
    elif round(float(result["effectValue"]["value"]), 3) >= 0.300 and round(float(result["effectValue"]["value"]), 3) <= 0.499:
        lstIVBatches['Batch6'] = lstIVBatches['Batch6'] + 1
        lstIVBatches['ObjBatch6'].append(npObj)  
    elif round(float(result["effectValue"]["value"]), 3) >= 0.500 and round(float(result["effectValue"]["value"]), 3) <= 1.000:
        lstIVBatches['Batch7'] = lstIVBatches['Batch7'] + 1
        lstIVBatches['ObjBatch7'].append(npObj)  

In [7]:
#########################################################################
# Convert dictionary to dataframe for further processing
dfDic = pd.DataFrame.from_dict(dicIVs, orient='index',
                       columns=['Batch1', 'Batch2', 'Batch3', 'Batch4', 'Batch5', 'Batch6', 'Batch7', 
                                'ObjBatch1', 'ObjBatch2', 'ObjBatch3', 'ObjBatch4', 'ObjBatch5',
                                         'ObjBatch6', 'ObjBatch7'])
#----------------------------------------------------------------------
# Sum the batches 
dfDic['sum'] = dfDic.apply(lambda row: row.Batch1 + row.Batch2 + row.Batch3 + row.Batch4 + row.Batch5 + row.Batch6 + row.Batch7, axis=1)
#--------------------
# Sort the data frame with respect to sum
dfDic.sort_values(by=['sum'], inplace=True, ascending=False)
df20 = dfDic.head(20)

# Now Implement the GUI

In [8]:
def getTop20_OfficialIVs():
    lstDropdown = df20.index.tolist()
    lstDropdown.insert(0, '-- Select All --')
    lstDropdown.append('primingConcept')
    lstDropdown.append('timePressure')
    lstDropdown.append('gameIncentive')
    lstDropdown.append('uncertaintyLevel')
    lstDropdown.append('anonymityManipul')
    return lstDropdown

In [9]:
%%HTML
<style>
.modal-content 
{
    min-width: 900px
}
</style>

javascript = """
<script type="text/Javascript">
    function showInfo(msg){
        var iframe = "<iframe width=\'854\' height=\'480\' src=\'np_r_html/"+msg+".html\' frameborder=\'0\' allowfullscreen></iframe>"
        require(
            ["base/js/dialog"], 
            function(dialog) {
              console.log(dialog)
                dialog.modal({
                    title: 'Nanopub Information',
                    body: $.parseHTML(iframe),
                    width: 'auto',
                    buttons: {
                        'Cancel': {}
                    }
                });
            }
        );
    }
</script>"""

HTML(javascript)

In [13]:
def drawCatplot():
    dicCatplot = {'IV': [], 'Batches':[], 'effectValues':[], 'stdErr': []}
    if dropdown.value == "-- Select All --":
        display(HTML("<h2> catplot about all IVs</h2>"))
        for key in getTop20_OfficialIVs():
            if key != '-- Select All --':
                for x in dicIVs[key].items():
                    if x[0]=='ObjBatch1':
                        for objNP in x[1]:
                            dicCatplot['IV'].append(key)
                            dicCatplot['Batches'].append('Large Negative Correlation')
                            dicCatplot['effectValues'].append(float(objNP['effectValue']))
                            dicCatplot['stdErr'].append(float(objNP['stdErr']))
                    elif x[0]=='ObjBatch2':
                        for objNP in x[1]:
                            dicCatplot['IV'].append(key)
                            dicCatplot['Batches'].append('Medium Negative Correlation')
                            dicCatplot['effectValues'].append(float(objNP['effectValue']))
                            dicCatplot['stdErr'].append(float(objNP['stdErr']))
                    elif x[0]=='ObjBatch3':
                        for objNP in x[1]:
                            dicCatplot['IV'].append(key)
                            dicCatplot['Batches'].append('Small Negative Correlation')
                            dicCatplot['effectValues'].append(float(objNP['effectValue']))
                            dicCatplot['stdErr'].append(float(objNP['stdErr']))
                    elif x[0]=='ObjBatch4':
                        for objNP in x[1]:
                            dicCatplot['IV'].append(key)
                            dicCatplot['Batches'].append('No Correlation')
                            dicCatplot['effectValues'].append(float(objNP['effectValue']))
                            dicCatplot['stdErr'].append(float(objNP['stdErr']))
                    elif x[0]=='ObjBatch5':
                        for objNP in x[1]:
                            dicCatplot['IV'].append(key)
                            dicCatplot['Batches'].append('Small Positive Correlation')
                            dicCatplot['effectValues'].append(float(objNP['effectValue']))
                            dicCatplot['stdErr'].append(float(objNP['stdErr']))
                    elif x[0]=='ObjBatch6':
                        for objNP in x[1]:
                            dicCatplot['IV'].append(key)
                            dicCatplot['Batches'].append('Medium Positive Correlation')
                            dicCatplot['effectValues'].append(float(objNP['effectValue']))
                            dicCatplot['stdErr'].append(float(objNP['stdErr']))
                    elif x[0]=='ObjBatch7':
                        for objNP in x[1]:
                            dicCatplot['IV'].append(key)
                            dicCatplot['Batches'].append('Large Positive Correlation')
                            dicCatplot['effectValues'].append(float(objNP['effectValue']))
                            dicCatplot['stdErr'].append(float(objNP['stdErr']))
    else:
        display(HTML("<h2> catplot about "+ dropdown.value +"</h2>"))
        key = dropdown.value
        for x in dicIVs[key].items():
            if x[0]=='ObjBatch1':
                for objNP in x[1]:
                    dicCatplot['IV'].append(key)
                    dicCatplot['Batches'].append('Large Negative Correlation')
                    dicCatplot['effectValues'].append(float(objNP['effectValue']))
                    dicCatplot['stdErr'].append(float(objNP['stdErr']))
            elif x[0]=='ObjBatch2':
                for objNP in x[1]:
                    dicCatplot['IV'].append(key)
                    dicCatplot['Batches'].append('Medium Negative Correlation')
                    dicCatplot['effectValues'].append(float(objNP['effectValue']))
                    dicCatplot['stdErr'].append(float(objNP['stdErr']))
            elif x[0]=='ObjBatch3':
                for objNP in x[1]:
                    dicCatplot['IV'].append(key)
                    dicCatplot['Batches'].append('Small Negative Correlation')
                    dicCatplot['effectValues'].append(float(objNP['effectValue']))
                    dicCatplot['stdErr'].append(float(objNP['stdErr']))
            elif x[0]=='ObjBatch4':
                for objNP in x[1]:
                    dicCatplot['IV'].append(key)
                    dicCatplot['Batches'].append('No Correlation')
                    dicCatplot['effectValues'].append(float(objNP['effectValue']))
                    dicCatplot['stdErr'].append(float(objNP['stdErr']))
            elif x[0]=='ObjBatch5':
                for objNP in x[1]:
                    dicCatplot['IV'].append(key)
                    dicCatplot['Batches'].append('Small Positive Correlation')
                    dicCatplot['effectValues'].append(float(objNP['effectValue']))
                    dicCatplot['stdErr'].append(float(objNP['stdErr']))
            elif x[0]=='ObjBatch6':
                for objNP in x[1]:
                    dicCatplot['IV'].append(key)
                    dicCatplot['Batches'].append('Medium Positive Correlation')
                    dicCatplot['effectValues'].append(float(objNP['effectValue']))
                    dicCatplot['stdErr'].append(float(objNP['stdErr']))
            elif x[0]=='ObjBatch7':
                for objNP in x[1]:
                    dicCatplot['IV'].append(key)
                    dicCatplot['Batches'].append('Large Positive Correlation')
                    dicCatplot['effectValues'].append(float(objNP['effectValue']))
                    dicCatplot['stdErr'].append(float(objNP['stdErr']))
                    
    dfCatplot = pd.DataFrame.from_dict(dicCatplot)
    sns.set_context("poster", font_scale = 0.7, rc={"grid.linewidth": 5})
    g = sns.catplot(x="effectValues", y="IV", hue="Batches", data=dfCatplot, ci='sd', palette="Set1", height=9, aspect=1.5)
    g.set_axis_labels('effectValues (-1 to 1)', 'Independent Variables')
    plt.show()
                    
def formatContent(npObj, IV, chk, dicSelectiveIV):
    splitNP = npObj['np'].split('/')
    splitObs = npObj['obs'].split('/')
    splitStudy = npObj['study'].split('/')
    oneShot = npObj['oneShot']
    matching = npObj['matching']
    gameIncentive = npObj['gameIncentive']
    Experimental = npObj['Experimental']
    yearOfDataCollection = npObj['yearOfDataCollection'].split(',')[0]
    authorCount = int(npObj['authorCount'])
    publicationYear = npObj['publicationYear']
    
    # Following are the integer values
    if npObj['numberOfChoices'] != '':
        numberOfChoices = int(statistics.median([int(e) for e in npObj['numberOfChoices'].split(',')]))
    else:
        numberOfChoices = ''
    
    if npObj['groupSize'] != '':
        groupSize = int(statistics.median([int(e) for e in npObj['groupSize'].split(',')]))
    else:
        groupSize = ''
    #===========================================================
    #Following are the float values
    if npObj['kIndex'] != '':
        kIndex = float(statistics.median([float(e) for e in npObj['kIndex'].split(',')]))
    else:
        kIndex = ''
        
    if npObj['MPCR'] != '':
        MPCR = float(statistics.median([float(e) for e in npObj['MPCR'].split(',')]))
    else:
        MPCR = ''
    
    if npObj['maleProportion'] != '':
        maleProportion = float(statistics.median([float(e) for e in npObj['maleProportion'].split(',')]))
    else:
        maleProportion = ''
        
    author = getAuthorFromDOI(npObj['doi'])
    if authorCount > 1:
         author = author + " et al.("+ str(publicationYear) +")"
    else:
        author = author + " ("+ str(publicationYear) +")"
            
    msg = urllib.parse.quote(splitNP[len(splitNP)-1])
    dicSelectiveIV['nano'].append('<span style="cursor:pointer" onclick="showInfo(\''+str(msg)+'\');">'+author+'</span>')
    dicSelectiveIV['iv'].append(IV)
    dicSelectiveIV['obs'].append(splitObs[len(splitObs)-1])
    dicSelectiveIV['Batches'].append(chk)
    dicSelectiveIV['effectValues'].append(float(npObj['effectValue']))
    dicSelectiveIV['stdErr'].append(float(npObj['stdErr']))
    dicSelectiveIV['study'].append(splitStudy[len(splitStudy)-1])
    dicSelectiveIV['oneShot'].append(oneShot)
    dicSelectiveIV['groupSize'].append(str(groupSize))
    #dicSelectiveIV['groupSizeFull'].append(npObj['groupSize'])
    dicSelectiveIV['matching'].append(matching)
    dicSelectiveIV['gameIncentive'].append(gameIncentive)
    dicSelectiveIV['Experimental'].append(Experimental)
    dicSelectiveIV['numberOfChoices'].append(str(numberOfChoices))
    #dicSelectiveIV['numberOfChoicesFull'].append(npObj['numberOfChoices'])
    
    dicSelectiveIV['kIndex'].append(str(kIndex))
    #dicSelectiveIV['kIndexFull'].append(npObj['kIndex'])
    dicSelectiveIV['MPCR'].append(str(MPCR))
    #dicSelectiveIV['MPCR'].append(npObj['MPCR'])
    dicSelectiveIV['maleProportion'].append(str(maleProportion))
    #dicSelectiveIV['maleProportion'].append(npObj['maleProportion'])
    dicSelectiveIV['yearOfDataCollection'].append(yearOfDataCollection)

#Contradiction Summary Dictionary
dictEvaluation = {}

def findContrastNano(dicSelectiveIV, df, contrast, IV):
    nano = dicSelectiveIV['nano']
    lstIndices = []
    lstIndicesCategory = []
    lstIndicesNumeric = []
    
    for cntrst in contrast:
        if cntrst == 'category':
            categoryCharacteristicsList = ['oneShot', 'matching', 'gameIncentive', 'Experimental']
            for categoryElement in categoryCharacteristicsList:
                categoryElementValues = dicSelectiveIV[categoryElement]
                if len(categoryElementValues) > 1:
                    my_dict = {i:categoryElementValues.count(i) for i in categoryElementValues}
                    if len(my_dict) > 0:
                        minval = min(my_dict.values())
                        if minval == 1:
                            minList = [k for k, v in my_dict.items() if v==minval]
                            for ele in minList:
                                index = categoryElementValues.index(ele)
                                lstIndicesCategory.append(str(index) +','+ str(df.index.tolist().index(categoryElement)))

        if cntrst == 'numeric':
            numericCharacteristicsList = ['yearOfDataCollection', 'numberOfChoices', 'groupSize', 'kIndex', 'MPCR', 'maleProportion']
            #numericCharacteristicsList = ['maleProportion']
            for numericElement in numericCharacteristicsList:
                numericElementValues = dicSelectiveIV[numericElement]
                my_dict = {i:numericElementValues.count(i) for i in numericElementValues}
                if len(my_dict) > 0:
                    minval = min(my_dict.values())
                    #print(my_dict)
                    #print(minval)
                    
                    if minval == 1:
                        minList = [k for k, v in my_dict.items() if v==minval]

                        if len(minList) == 1:
                            #print(minList)
                            index = numericElementValues.index(minList[0])
                            lstIndicesNumeric.append(str(index) +','+ str(df.index.tolist().index(numericElement)))
                        else:
                            minList =  list(filter(None, minList))
                            #print(minList)

                            if numericElement in ['kIndex', 'MPCR', 'maleProportion']:
                                maxValue = max([float(i) for i in minList]) #float(max(minList))
                                minValue = min([float(i) for i in minList]) #float(min(minList))
                            else:
                                maxValue = max([int(i) for i in minList]) #int(max(minList))
                                minValue = min([int(i) for i in minList]) #int(min(minList))

                            maxValueList = []
                            minValueList = []
                            
                            #print(minValue, maxValue)
                            for ele in minList:
                                #print(ele)
                                if numericElement in ['kIndex', 'MPCR', 'maleProportion']:
                                    if (float(ele) - minValue) > (maxValue - float(ele)):
                                        maxValueList.append(ele)
                                    else:
                                        minValueList.append(ele)
                                else:
                                    if (int(ele) - minValue) > (maxValue - int(ele)):
                                        maxValueList.append(ele)
                                    else:
                                        minValueList.append(ele)

                            if len(minValueList) < len(maxValueList):
                                for k in minValueList:
                                    index = numericElementValues.index(k)
                                    lstIndicesNumeric.append(str(index) +','+ str(df.index.tolist().index(numericElement)))
                            else:
                                for k in maxValueList:
                                    index = numericElementValues.index(k)
                                    lstIndicesNumeric.append(str(index) +','+ str(df.index.tolist().index(numericElement)))

                            #print(maxValueList)
                            #print(minValueList)
    totalNanoCategory = []
    totalNanoNumeric = []
    for catIndex in lstIndicesCategory:
        splitCatIndices = catIndex.split(',')
        if splitCatIndices[0] not in totalNanoCategory:
            totalNanoCategory.append(splitCatIndices[0])
            
    for numIndex in lstIndicesNumeric:
        splitNumIndices = numIndex.split(',')
        if splitNumIndices[0] not in totalNanoNumeric:
            totalNanoNumeric.append(splitNumIndices[0])
            
    lstIndices = lstIndicesCategory + lstIndicesNumeric
    print('Summary\n-------------')
    print('Total Nanopubs: %s' % len(nano))
    print('Total Nanopubs highlighted by Category: %s' % len(totalNanoCategory))
    print('Total Nanopubs highlighted by Numeric: %s' % len(totalNanoNumeric))
    
    dictEvaluation[IV] = {'Categorical': len(totalNanoCategory), 'Numerical': len(totalNanoNumeric)}
    
    #print(lstIndices)
    return lstIndices

def color(x,row_idx,col_idx):
    #print(x)
    #for index, row in x.iterrows():
    #    print(row[4][4])
    #print(row_idx)
    #print(col_idx)
    #effectValues
    #Old Formulla = str(((float(shade[colIndex])/2)*10))
    #New hue formulla
    #hue = effectValue / ((stdErr/2)*10)
    
    shade = []
    for col in x.columns:
        stdErr = x[col].iloc[x.index.tolist().index('stdErr')]
        effectVal = x[col].iloc[x.index.tolist().index('effectValues')]
        if float(effectVal) < 0:
            effectVal = effectVal * -1
            
        hue = 1 - (float(stdErr)/2)*10  #(float(effectVal) / ((float(stdErr)/2)*10))/10
        shade.append(str(hue))
    
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    
    for colIndex in range(len(x.columns)):
        df_styler.iloc[3, colIndex] = 'background-color: rgba(255,0,0,'+shade[colIndex]+');'
    
    colIndexList = []
    rowIndexList = []
    for idx in range(len(col_idx)):
        if col_idx[idx] not in colIndexList:
            colIndexList.append(col_idx[idx])
            
        if row_idx[idx] not in rowIndexList:
            rowIndexList.append(row_idx[idx])
    
    for iCol in colIndexList:
        for iRow in range(len(x.index)):
            if iRow == 3:
                df_styler.iloc[iRow, iCol] = 'background-color: rgba(255,0,0,'+shade[colIndex]+');border-left:3px solid black;border-right:3px solid black;'
            else:
                df_styler.iloc[iRow, iCol] = 'border-left:3px solid black;border-right:3px solid black;'
            
    for iRow in rowIndexList:
        for iCol in range(len(x.columns)):
            df_styler.iloc[iRow, iCol] = 'border-top:3px solid black;border-bottom:3px solid black;'
    
    color = ''
    for idx in range(len(col_idx)):
        color = 'background-color: rgba(255,0,0,'+shade[colIndex]+');border-left:3px solid black;border-right:3px solid black;border-top:3px solid black;border-bottom:3px solid black'
        df_styler.iloc[row_idx[idx], col_idx[idx]] = color
    
        
    for idx in range(len(col_idx)):
        value = x.iloc[row_idx[idx], col_idx[idx]]
        #msg = urllib.parse.quote(x.columns[col_idx[idx]].split('>')[1].split('</')[0])
        msg = x.columns[col_idx[idx]].split('(')[1].split(')')[0]
        x.iloc[row_idx[idx], col_idx[idx]] = '<span style="cursor:pointer" onclick="showInfo(\''+str(msg)+'\');">'+value+'</span>'
        
    #print(df_styler)
    return df_styler

def on_button_clicked(b):
    button.description = 'Please wait!'
    button.disabled = True
    dicSelectiveIV = { 'nano':[], 'iv':[], 'obs':[], 'Batches':[], 'effectValues':[], 'stdErr': [], 'study': [],
                    'oneShot': [],
                    'groupSize': [],
                    #'groupSizeFull': [],  
                    'matching': [],
                    'gameIncentive': [],
                    'Experimental': [],
                    'numberOfChoices': [],
                    #'numberOfChoicesFull':[],
                    'kIndex': [],
                    'MPCR': [],
                    'maleProportion': [],
                    'yearOfDataCollection': []
                 }
    
    #print(dropdown.value)
    contrastOption = []
    if checkboxCategory.value == True:
        contrastOption.append('category')
    if checkboxNumeric.value == True:
        contrastOption.append('numeric')
        
    if dropdown.value == "-- Select All --":
        clear_output(True)
        createGUI()
        
        dictEvaluation = {}
        
        # To draw the catplot by Independent variable
        if checkboxCatplot.value == True:
            drawCatplot()
            
        for iv in getTop20_OfficialIVs():
            if iv != '-- Select All --':
                dicSelectiveIV = { 'nano':[], 'iv':[], 'obs':[], 'Batches':[], 'effectValues':[], 'stdErr': [], 'study': [],
                    'oneShot': [],
                    'groupSize': [],
                    #'groupSizeFull': [],  
                    'matching': [],
                    'gameIncentive': [],
                    'Experimental': [],
                    'numberOfChoices': [],
                    #'numberOfChoicesFull':[],
                    'kIndex': [],
                    'MPCR': [],
                    'maleProportion': [],
                    'yearOfDataCollection': []
                 }
                
                #Select batches according to the selection of check boxes
                title = ''
                if checkboxLNP.value == True or checkboxAllBatches.value == True:
                    title = 'Large, '
                    #Large Negative
                    nps = dicIVs[iv]['ObjBatch1']
                    for objNP in nps:
                        formatContent(objNP, iv, 'Large Negative Correlation', dicSelectiveIV)
                    
                    #Large Positive
                    nps = dicIVs[iv]['ObjBatch7']
                    for objNP in nps:
                        formatContent(objNP, iv, 'Large Positive Correlation', dicSelectiveIV)
                        
                if checkboxMNP.value == True or checkboxAllBatches.value == True:
                    title += 'Medium, '
                    #Medium Negative
                    nps = dicIVs[iv]['ObjBatch2']
                    for objNP in nps:
                        formatContent(objNP, iv, 'Medium Negative Correlation', dicSelectiveIV)
                    
                    #Medium Positive
                    nps = dicIVs[iv]['ObjBatch6']
                    for objNP in nps:
                        formatContent(objNP, iv, 'Medium Positive Correlation', dicSelectiveIV)
                        
                if checkboxSNP.value == True or checkboxAllBatches.value == True:
                    title += 'Small, '
                    #Small Negative
                    nps = dicIVs[iv]['ObjBatch3']
                    for objNP in nps:
                        formatContent(objNP, iv, 'Small Negative Correlation', dicSelectiveIV)
                    
                    #Small Positive
                    nps = dicIVs[iv]['ObjBatch5']
                    for objNP in nps:
                        formatContent(objNP, iv, 'Small Positive Correlation', dicSelectiveIV)
                
                
                if checkboxAllBatches.value == True:
                    title = 'Large, Medium, Small (Negative & Positive) and No Correlation '
                    #No Correlation
                    nps = dicIVs[iv]['ObjBatch4']
                    for objNP in nps:
                        formatContent(objNP, iv, 'No Correlation', dicSelectiveIV)
                        
                title = title.strip()
                title = title.rstrip(",") + ' (Negative & Positive) Correlation '
                #########################
                r_idx = []
                c_idx = []
                
                dfSelectiveIV = pd.DataFrame.from_dict(dicSelectiveIV, orient='index', columns=dicSelectiveIV['nano'])
                display(HTML("<h2> "+ title + "for " + iv +"</h2>"))
                highlightNano = findContrastNano(dicSelectiveIV, dfSelectiveIV, contrastOption, iv)
                dfSelectiveIV = dfSelectiveIV.drop(['nano'])
                if len(highlightNano) == 0 or checkboxHighlight.value == False:
                    display(dfSelectiveIV.style.apply(color, row_idx = r_idx, col_idx = c_idx, axis = None))
                else:
                    for index in highlightNano:
                        splitIndex = index.split(',')
                        r_idx.append(int(splitIndex[1]) - 1)
                        c_idx.append(int(splitIndex[0]))

                    display(dfSelectiveIV.style.apply(color, row_idx = r_idx, col_idx = c_idx, axis = None))
                    
                display(HTML("<hr />"))
    else:
        clear_output(True)
        createGUI()
        dictEvaluation = {}
        
        # To draw the catplot by Independent variable
        if checkboxCatplot.value == True:
            drawCatplot()
        
        #Select batches according to the selection of check boxes
        title = ''
        if checkboxLNP.value == True or checkboxAllBatches.value == True:
            #display(HTML("<p>IN</p>"))
            title = 'Large, '
            #Large Negative
            nps = dicIVs[dropdown.value]['ObjBatch1']
            for objNP in nps:
                formatContent(objNP, dropdown.value, 'Large Negative Correlation', dicSelectiveIV)
                
            #Large Positive
            nps = dicIVs[dropdown.value]['ObjBatch7']
            for objNP in nps:
                formatContent(objNP, dropdown.value, 'Large Positive Correlation', dicSelectiveIV)
            
                
        if checkboxMNP.value == True or checkboxAllBatches.value == True: 
            title += 'Medium, '
            #Medium Negative
            nps = dicIVs[dropdown.value]['ObjBatch2']
            for objNP in nps:
                formatContent(objNP, dropdown.value, 'Medium Negative Correlation', dicSelectiveIV)
            
            #Medium Positive
            nps = dicIVs[dropdown.value]['ObjBatch6']
            for objNP in nps:
                formatContent(objNP, dropdown.value, 'Medium Positive Correlation', dicSelectiveIV)
                
        if checkboxSNP.value == True or checkboxAllBatches.value == True: 
            title += 'Small, '
            #Small Negative
            nps = dicIVs[dropdown.value]['ObjBatch3']
            for objNP in nps:
                formatContent(objNP, dropdown.value, 'Small Negative Correlation', dicSelectiveIV)
            
            #Small Positive
            nps = dicIVs[dropdown.value]['ObjBatch5']
            for objNP in nps:
                formatContent(objNP, dropdown.value, 'Small Positive Correlation', dicSelectiveIV)
            
        if checkboxAllBatches.value == True:
            title = 'Large, Medium, Small and No '
            #No Correlation
            nps = dicIVs[dropdown.value]['ObjBatch4']
            for objNP in nps:
                formatContent(objNP, dropdown.value, 'No Correlation', dicSelectiveIV)
                
        
        title = title.strip()
        title = title.rstrip(",") + ' (Negative & Positive) Correlation '
        ############################
        r_idx = []
        c_idx = []
        dfSelectiveIV = pd.DataFrame.from_dict(dicSelectiveIV, orient='index', columns=dicSelectiveIV['nano'])
        display(HTML("<h2> "+ title + "for " + dropdown.value +"</h2>"))
        highlightNano = findContrastNano(dicSelectiveIV, dfSelectiveIV, contrastOption, dropdown.value)
        dfSelectiveIV = dfSelectiveIV.drop(['nano'])
        if len(highlightNano) == 0 or checkboxHighlight.value == False:
            display(dfSelectiveIV.style.apply(color, row_idx = r_idx, col_idx = c_idx, axis = None))
        else:
            for index in highlightNano:
                splitIndex = index.split(',')
                r_idx.append(int(splitIndex[1]) - 1)
                c_idx.append(int(splitIndex[0]))
                
            display(dfSelectiveIV.style.apply(color, row_idx = r_idx, col_idx = c_idx, axis = None))
    
    #Button Enabled
    button.description = 'Submit'
    button.disabled = False

In [14]:
#Define Controls
dropdown = widgets.Dropdown(
    options= getTop20_OfficialIVs(),
    value='-- Select All --',
    description='IVs:',
    disabled=False,
)

label = widgets.HTML(
    value="<hr /><b>Please select the batches</b>"
)

checkboxLNP = widgets.Checkbox(
    value=True,
    description='Large Negative & Positive Correlation',
    disabled=False,
    indent=False
)
checkboxMNP = widgets.Checkbox(
    value=False,
    description='Medium Negative & Positive Correlation',
    disabled=False,
    indent=False
)

checkboxSNP = widgets.Checkbox(
    value=False,
    description='Small Negative & Positive Correlation',
    disabled=False,
    indent=False
)

checkboxAllBatches = widgets.Checkbox(
    value=False,
    description='All Correlation',
    disabled=False,
    indent=False
)

labelLine = widgets.HTML(
    value="<hr /><b>Nanopub Contrast Analysis</b>"
)


checkboxHighlight = widgets.Checkbox(
    value=True,
    description='Highlight Contrast Cells?',
    disabled=False,
    indent=False
)

checkboxCategory = widgets.Checkbox(
    value=True,
    description='By Category?',
    disabled=False,
    indent=False
)

checkboxNumeric = widgets.Checkbox(
    value=False,
    description='By Numeric values?',
    disabled=False,
    indent=False
)

labelExtra = widgets.HTML(
    value="<hr /><b>Extra</b>"
)

checkboxCatplot = widgets.Checkbox(
    value=False,
    description='Do you want to view catplot?',
    disabled=False,
    indent=False
)

button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submit',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

def createGUI():
    button.on_click(on_button_clicked)
    items = [dropdown, label]
    itemsChk1 = [ checkboxLNP, checkboxMNP]
    itemsChk2 = [ checkboxSNP, checkboxAllBatches]
    itemContrast = [checkboxCategory, checkboxNumeric]
    display(widgets.VBox ([widgets.VBox(items), widgets.HBox(itemsChk1), widgets.HBox(itemsChk2) , labelLine, 
                           checkboxHighlight, widgets.HBox(itemContrast), labelExtra, checkboxCatplot, button]))

In [15]:
createGUI()

In [12]:
dfEvaluation = pd.DataFrame.from_dict(dictEvaluation, orient='index')
#dfEvaluation